In [ ]:
!pip install spacy
!pip install newsapi-python
!pip install wordcloud
!python -m spacy download en_core_web_lg

In [ ]:
from newsapi import NewsApiClient
import spacy
import pickle

API_KEY = '3fce5faf7272425ba4fc57f8d94d5f4e'
# nlp_eng = spacy.load("en_core_web_sm")
nlp_eng = spacy.load('en_core_web_lg')
news_api = NewsApiClient(api_key=API_KEY)

In [ ]:
import pandas as pd

pageCount = 5
totalResult = 0
articles=[]

for curPage in range(pageCount):
  temp = news_api.get_everything(q='coronavirus',
                               language='en',
                               sort_by='relevancy',
                               page=curPage+1)
  for item in temp['articles']:
    articles.append(item)
  totalResult += len(temp['articles'])

print('total search results: %d' % totalResult)
print(articles)

# save the articles dataset using pickle
filename = 'articlesCOVID.pckl'
pickle.dump(articles, open(filename, 'wb'))

filename = 'articlesCOVID.pckl'
loaded_articles= pickle.load(open(filename, 'rb'))
print(loaded_articles)
print(len(loaded_articles))

# save the articles dataset with parsed information using pandas
data = {
    'author': [],
    'title': [],
    'description': [],
    'content': []
}
for _, article in enumerate(loaded_articles):
  author = article['author']
  title = article['title']
  desc = article['description']
  content = article['content']

  data['author'].append(author)
  data['title'].append(title)
  data['description'].append(desc)
  data['content'].append(content)

df = pd.DataFrame(data)
df.head()

df.to_excel('covid_articles.xlsx', index = True, header=True)
print (df)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import string

def get_keywords_eng(text, nlp):
  result = []
  pos_tag = ['PROPN', 'VERB', 'NOUN'] 
  doc = nlp(text)
  for token in doc:
    if token.text in nlp_eng.Defaults.stop_words or token.text in string.punctuation:
      continue          # exclude all the STOPWORDS and punctuation from the list
    elif token.pos_ in pos_tag:
      result.append(token.text)
  return result

def plot_word_cloud(word_cloud):
  plt.figure(figsize=(20,15))
  plt.imshow(word_cloud)
  plt.axis('off')
  plt.show()

def generate_word_cloud(articles, type): # type can only be ['title', 'description', 'content']
  results = []
  
  # iterate over 100 results from search
  for _, article in enumerate(articles):
    item = article[type]
    if item is None:
      continue
    
    # get the most common keywords from each article
    for keyword in Counter(get_keywords_eng(item, nlp_eng)).most_common(5):
      results.append(keyword[0])
    
  text = ' '.join(results)
  wordcloud = WordCloud(max_font_size=50, max_words=100, collocations=False,
                      background_color='salmon', colormap='Pastel1').generate(text)       # collocations=False for no duplications
  return wordcloud

In [ ]:
total_rows = 2
total_cols = 2
row = 0
col = 0

fig, ax = plt.subplots(total_rows, total_cols,figsize=(20, 10))
for type in ['title', 'description', 'content']:
  # generate WordCloud from articles
  wordcloud = generate_word_cloud(articles, type)

  # display the WordCloud in the figure
  plt.rcParams.update({'font.size': 16})
  ax[row][col].imshow(wordcloud, interpolation='bilinear')

  # do not display axis of subplots
  ax[row][col].axis('off')

  # display title for each subplot
  plt.rcParams.update({'font.size': 22})
  ax[row][col].set_title('WordCloud for Coronavirus using \'%s\'' % type)
  plt.rcParams.update({'font.size': 16})

  # update subplot position
  row = row + 1
  if row == total_rows:
    row = 0
    col = col + 1

ax[1][1].axis('off')
plt.tight_layout(pad=1)
plt.show()